In [112]:
import yaml
import pandas as pd
import os
from datetime import datetime
import mysql.connector
# Dictionary of folder paths
folder_path_main = {
    "folder_path_1": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2023-10",
    "folder_path_2": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2023-11",
    "folder_path_3": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2023-12",
    "folder_path_4": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-01",
    "folder_path_5": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-02",
    "folder_path_6": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-03",
    "folder_path_7": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-04",
    "folder_path_8": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-05",
    "folder_path_9": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-06",
    "folder_path_10": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-07",
    "folder_path_11": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-08",
    "folder_path_12": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-09",
    "folder_path_13": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-10",
    "folder_path_14": "C:/Users/NAVEEN/OneDrive/Desktop/extract_Data/2024-11",
}
#create the function with arugemnt(folder) for getting all the values in folder_path_main(dictinoary)
def get_folder_path(folder):
    folder_path = folder_path_main[folder]
    #using try block to avoid the error (yaml contains unorder date)
    try:
        # Get all files in the folder
        items = os.listdir(folder_path) 
        # Initialize an empty DataFrame 
        month = pd.DataFrame()  
        #using for loop for iterate the each yaml file in every single folder
        for i in items:
            # file_path(Correct variable) to joining the each yaml file with folder path
            file_path = os.path.join(folder_path, i) 
            # To the Read the file_path  
            with open(file_path, 'r') as file:
                # To load the yaml file
                data = yaml.safe_load(file)
                df = pd.DataFrame(data)
                # Correct order of concatenation # Merge DataFrames
                month = pd.concat([month, df], ignore_index=True)  

        return month  # Return the final DataFrame

    except Exception as e:
        print(f"Error: {e}")
        return None  # Return None in case of an error

# get the values of each folder_path and store in varible(result)
result_1 = get_folder_path("folder_path_1")
result_2 = get_folder_path("folder_path_2")
result_3 = get_folder_path("folder_path_3")
result_4 = get_folder_path("folder_path_4")
result_5 = get_folder_path("folder_path_5")
result_6 = get_folder_path("folder_path_6")
result_7 = get_folder_path("folder_path_7")
result_8 = get_folder_path("folder_path_8")
result_9 = get_folder_path("folder_path_9")
result_10 = get_folder_path("folder_path_10")
result_11 = get_folder_path("folder_path_11")
result_12 = get_folder_path("folder_path_12")
result_13 = get_folder_path("folder_path_13")
result_14 = get_folder_path("folder_path_14")
# to contact the each result as stored as main_result
main_result =  pd.concat([result_1, result_2,result_3,result_4,result_5,result_6,result_7,result_8,result_9,result_10,result_11,result_12,result_13,result_14,], ignore_index=True) 

#  using for loop to iterate all unique values in Ticker cloumn 
for ticker in main_result["Ticker"].unique():
    df_ticker = main_result[main_result["Ticker"] == ticker]
    # To save categoery based Ticker as csv_files for future analysis
    df_ticker.to_csv(f"{ticker}.csv", index=False)
    print(f"Saved {ticker}.csv")



# Creating a list to store each ticker's DataFrame
df_list = []

# using for loop to iterate all unqiue values in Ticker column
for ticker in main_result["Ticker"].unique():
    df_ticker = main_result[main_result["Ticker"] == ticker]
    df_list.append(df_ticker)  # Add each filtered DataFrame to the list

# Concatenate all DataFrames into one
final_df = pd.concat(df_list, ignore_index=True)

# Save to a single CSV file
final_df.to_csv("All_Tickers.csv", index=False)

#To read the csv file
final_stock  = pd.read_csv("All_Tickers.csv")

#To convert the date column is in datetime format
final_stock['date'] = pd.to_datetime(final_stock['date'])

# Exract year from the date column
final_stock['year'] = final_stock['date'].dt.year

# Exract year from the date column
final_stock['months'] = final_stock['date'].dt.month

# Exract year from the date column
final_stock['day'] = final_stock['date'].dt.day



#Year wise anlaysis part

#To calculate the ticker_wise_stock for year_wise(2023_2024)for each Ticker

# To create the List
year_price = []
tickers = []
#using for loop to calculate open and closing stock year wise
for i in final_stock['Ticker'].unique():
    # using this variable(Ticker_unique) to get each Ticker values
    Ticker_unique = final_stock[final_stock['Ticker'] == i]
    #using this varaible(year_trend)to calculate the value for yearwise using iloc function(row_wise_index)
    year_trend = ((Ticker_unique.iloc[-1]['close'] - Ticker_unique.iloc[0]['open'])/Ticker_unique.iloc[-1]['open'])*100
    #To append the ticker_value
    tickers.append(i)
    # To append the year_trend
    year_price.append(year_trend) 

# To creat the dataframe 
year_data = pd.DataFrame({'Ticker': tickers, 'stock_price_year': year_price})

# Top_10 Green_Stcoks for year_wise
Top_10_Green_Stocks = year_data.sort_values(by= 'stock_price_year', ascending=False).head(10)
#To reset the index value 
Top_10_Green_Stocks = Top_10_Green_Stocks.reset_index(drop=True)

# Save to a single CSV file
Top_10_Green_Stocks.to_csv("Top_10_Green_Stocks", index=False)

# Top_10 Loss_ Stcoks for year_wise
Top_10_Loss_Stocks = year_data.sort_values(by='stock_price_year', ascending=False).tail(10)

#To reset the index value 
Top_10_Loss_Stocks = Top_10_Loss_Stocks.reset_index(drop=True)

# Save to a single CSV file
Top_10_Loss_Stocks.to_csv("Top_10_Loss_Stocks", index=False)

#Average volume for Each strock over the Year
Avg_Volume_per_stocks = final_stock.groupby("Ticker")["volume"].mean().round(2).reset_index()
Avg_Volume_per_stocks = Avg_Volume_per_stocks.rename(columns={"volume":"Avg_volume"})

# Save to a single CSV file
Avg_Volume_per_stocks.to_csv("Avg_Volume_per_stocks", index=False)


#Average Price for Each strock over the Year
Avg_prices_per_stocks = final_stock.groupby("Ticker")["close"].mean().reset_index()
Avg_prices_per_stocks = Avg_prices_per_stocks.rename(columns={"close": "Avg_Close_Price"})

# Save to a single CSV file
Avg_prices_per_stocks.to_csv("Avg_prices_per_stocks", index=False)

# To calculate the Number of Green and Red stcoks over the Year
green_stocks = (final_stock["close"] > final_stock["open"]).sum()
red_stocks = (final_stock["close"] < final_stock["open"]).sum()


# Creating a DataFrame with an index
No_green_red_stocks = pd.DataFrame({
            "Stock Type": ["Green Stocks", "Red Stocks"],
            "Count": [green_stocks, red_stocks]})


# Save to a single CSV file
No_green_red_stocks.to_csv("No_green_red_stocks", index=False)


#  Volatility Analysis part

#To create the New column using the prevoius column with shift operation
final_stock['pre_close'] = final_stock['close'].shift(1)

#using For loop to do calculate daily Return for each stock in single day
for i in final_stock["Ticker"]:
    final_stock['daily_return'] = (final_stock['close'] - final_stock['pre_close']) / final_stock['pre_close']

# After that i will check any null values in my dataset

# we have 1(Null_values) in my first Row of dataset (final_Stock.iloc[0])

#so i will drop my null values using dropna method
final_stock.dropna(inplace=True)

#To calculate the mean value for daily_return for each Tickers(stocks)
#using reset_index to get proper index values
AVG_per_Stocks_day = final_stock.groupby('Ticker')["daily_return"].mean().reset_index()

#Top 10 Most volatile stocks over the year using head function
Top_10_Most_Volatile_Stocks = AVG_per_Stocks_day.sort_values("daily_return",ascending=False).head(10).reset_index(drop=True)

# Save to a single CSV file
Top_10_Most_Volatile_Stocks.to_csv("Top_10_Most_Volatile_Stocks", index=False)


#Comulative Analysis part

#To create the list 
comulative_return = []
comulative_ticker = []

#using For loop for each Ticker values to calculate the comulatuive values
for i in final_stock["Ticker"].unique():
    com_Ticker = final_stock[final_stock["Ticker"] == i]

    #using iloc method to choose specific rows in single cloumn("close") in dataframe
    com_return = (com_Ticker.iloc[0]["close"] - com_Ticker.iloc[-1]["close"]) / com_Ticker.iloc[0]["close"]*100
    #To append the values in the list
    comulative_ticker.append(i)  
    comulative_return.append(com_return)  

#To create the variable for the process of dataframe 
comulative_return_year = pd.DataFrame({
    "Ticker": comulative_ticker, 
    "Cumulative Return": comulative_return
})

# Save to a single CSV file
comulative_return_year.to_csv("comulative_return_year", index=False)


# stock price correlation part

# Pivot Data: Each stock has a separate column for 'close' price
df_pivot = final_stock.pivot(index="date", columns="Ticker", values="close")

# Compute the correlation matrix
correlation_matrix = df_pivot.corr()

# Taking corrlation matrix column values for the fliteration
columns = correlation_matrix.columns
 # Create the list
data = []
 # Using For loop for iterate both rows and values
for i,row in correlation_matrix.iterrows():
        for cloumn in columns :
            #To add the values in list with dictionary format
            data.append({
                    "row" : i,
                    "cloumn" : cloumn,
                    "corr" : row[cloumn]


            })

#To create the Dataframe
corr_data = pd.DataFrame(data)

        
# Drop rows where corr = 1 (Self-correlation)
corr_data = corr_data[corr_data["corr"] != 1.000000]


 # Function to categorize correlation values
def categorize_correlation(value):
    if value >= 0.8:
        return "Positive Correlation"
    elif 0.5 <= value < 0.8:
        return "Moderate Correlation"
    elif -0.5 < value < 0.5:
        return "No Correlation"
    else:
        return "Negative Correlation"
    

# Apply categorization function
corr_data["Category"] = corr_data["corr"].apply(categorize_correlation)

# Save to a single CSV file
corr_data.to_csv("correlation_matrix", index=False)


# Sector Wise Analysis part 

#To Read the csv file
sector = pd.read_csv("C:/Users/NAVEEN/Downloads/Sector_data - Sheet1 (2).csv")

#from this csv we can drop the cloumn(company), the cloumn not use in feature pupurose
sector.drop("COMPANY",axis=1,inplace=True)
# To Rename the cloumn name for Feature pupurose(do the function two dataframe)
sector.rename(columns={"Symbol":"Ticker"},inplace=True)

#using split function to split data in ticker cloumn
sector["Ticker"] = sector["Ticker"].str.split(":").str[1].str.strip()



# Initialize an empty dictionary
sec_ = {} 
# Iterate over the DataFrame rows
for i, row in sector.iterrows():  
# Assign values correctly
     sec_[row[1]] = row[0] 


# To create the cloumn in dataframe and using (None) to avoid the key error
final_stock["sector"] = None  

#using For loop to iterate the ticker values for mapping
for i in final_stock["Ticker"]:

    # The iterate values is avaliable in dictionary(sec_)
    if i in sec_: 
        final_stock.loc[final_stock["Ticker"] == i, "sector"] = sec_[i]
    else:
        print(f"Warning: '{i}' not found in sector mapping")


# Initialize lists to store results
overall_returns = []
sectors = []
tickers = []

# Loop through unique Ticker-Sector combinations
for ticker in final_stock["Ticker"].unique():
    for sector in final_stock["sector"].unique():
        
        # Filter data for the given Ticker and Sector across all years
        overall_data = final_stock[(final_stock["Ticker"] == ticker) & (final_stock["sector"] == sector)]
        
        # Check if there are at least two rows 
        try:
            # Calculate overall return from first available close to last available close
            overall_return = ((overall_data.iloc[-1]["close"] - overall_data.iloc[0]["close"]) / overall_data.iloc[0]["close"]) * 100
            
            # Append values to lists
            tickers.append(ticker)
            sectors.append(sector)
            overall_returns.append(overall_return)

        # Skip this iteration if there's not enough data
        except:
            continue

# Create DataFrame for results
sector_wise_overall_return = pd.DataFrame({
    "Ticker": tickers,
    "Sector": sectors,
    "Overall Return (2023-2024)": overall_returns
    })


# Save to a single CSV file
sector_wise_overall_return.to_csv("sector_wise_overall _return", index=False)


# Top 5 Month_Wise_Gainers and Losers part

# Compute monthly returns
final_stock["month_return"] = ((final_stock["close"] - final_stock["open"]) / final_stock["open"]) * 100

# Group by Ticker and Month
month_wise_return = final_stock.groupby(["Ticker", "month"])["month_return"].mean().reset_index()


# Save to a single CSV file
month_wise_return.to_csv("Month_wise_Gainers&Losers", index=False)



# datawarehousing part

# connection vscode to mysql(database)
try:

    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Naveen@090194"
    )
    mycursor = mydb.cursor()
    
    # creating the database in mysql
    mycursor.execute("CREATE DATABASE stock_analysis")

    print("databse succesfully created in mysql")

except :
    print(" the database already created")

#create the table for in database(stock_analysis)

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query = ('''CREATE TABLE stock_details (     Ticker  VARCHAR(255), 
                                                        close   float,
                                                        date   DATETIME,
                                                        high    float,
                                                        low     float,
                                                        month   VARCHAR(7),
                                                        open    float,
                                                        volume  int,
                                                        year    int,
                                                        months  int,
                                                        day     int,
                                                        pre_close float,
                                                       	daily_return float,
                                                      	sector VARCHAR(255),
                                                        month_return float,
                                                        PRIMARY KEY (Ticker, date) 
                                                        )''')
    mycursor.execute(create_query)

    print(" stock_details TABLES ARE SUCCESFUULY CREATED IN MYSQL")
except :
  print(" stock_details TABLES ALREADY  IN MYSQL")


# To get the values in dataframe using (.values) and converting to list
insert_values = final_stock.values.tolist()

#To store in values in the stcok_details table
try:

    insert_query = ''' insert into stock_details 
                     values
                     (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
    mycursor.executemany(insert_query,insert_values)   
    mydb.commit()
    print(" stock_details values are successfully inserted in the table")
#To avoid the duplicate entry so we can use try and except method    
except :
    print(" stock_details values are already exist")



#create the table for in database(Top_10_green_stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_1 = ('''CREATE TABLE Top_10_green_stocks( Ticker  VARCHAR(255), 
                                                            stock_price_year float)''')
    
    mycursor.execute(create_query_1)

    print("Top_10_Green_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("Top_10_Green_Stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
Top_10_green_values = Top_10_Green_Stocks.values.tolist()


#To store in values in the Top_10_green_stocks
try:

    insert_query_1 = ''' insert into  Top_10_green_stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_1,Top_10_green_values)   
    mydb.commit()
    print("Top_10_Green_Stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("Top_10_Green_Stocks values are already exist")



#create the table for in database(Top_10_Loss_Stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_2 = ('''CREATE TABLE Top_10_Loss_Stocks( Ticker  VARCHAR(255), 
                                                            stock_price_year float)''')
    
    mycursor.execute(create_query_2)

    print("Top_10_Loss_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("Top_10_Loss_Stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
Top_10_loss_values = Top_10_Loss_Stocks.values.tolist()


#To store in values in the Top_10_Loss_Stocks
try:

    insert_query_2 = ''' insert into Top_10_Loss_Stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_2,Top_10_loss_values)   
    mydb.commit()
    print("Top_10_Loss_Stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("Top_10_Loss_Stocks values are already exist")



#create the table for in database(Avg_prices_per_stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_3 = ('''CREATE TABLE Avg_prices_per_stocks( Ticker  VARCHAR(255), 
                                                              Avg_Close_Price float)''')
    
    mycursor.execute(create_query_3)

    print("Avg_prices_per_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("Avg_prices_per_stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
Average_price_values = Avg_prices_per_stocks.values.tolist()


#To store in values in the Avg_prices_per_stocks
try:

    insert_query_3 = ''' insert into  Avg_prices_per_stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_3,Average_price_values)   
    mydb.commit()
    print("Avg_prices_per_stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("Avg_prices_per_stocks values are already exist")




#create the table for in database(Avg_Volume_per_stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_4 = ('''CREATE TABLE Avg_Volume_per_stocks( Ticker  VARCHAR(255), 
                                                              Avg_volume_Price float)''')
    
    mycursor.execute(create_query_4)

    print("Avg_Volume_per_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("Avg_Volume_per_stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
Average_volume_values=Avg_Volume_per_stocks.values.tolist()


#To store in values in the Avg_Volume_per_stocks
try:

    insert_query_4 = ''' insert into  Avg_Volume_per_stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_4,Average_volume_values)   
    mydb.commit()
    print("Avg_Volume_per_stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("Avg_Volume_per_stocks values are already exist")



#create the table for in database(No_green_red_stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_5 = ('''CREATE TABLE No_green_red_stocks( Stock_Type  VARCHAR(255), 
                                                           Count int)''')
    
    mycursor.execute(create_query_5)

    print("No_green_red_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("No_green_red_stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
No_green_red_values = No_green_red_stocks.values.tolist()


#To store in values in the No_green_red_stocks table
try:

    insert_query_5 = ''' insert into No_green_red_stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_5,No_green_red_values)   
    mydb.commit()
    print("No_green_red_stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("No_green_red_stocks values are already exist")



#create the table for in database(Top_10_Most_Volatile_Stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_6 = ('''CREATE TABLE Top_10_Most_Volatile_Stocks( Ticker VARCHAR(255), 
                                                           volatile_retutn float)''')
    
    mycursor.execute(create_query_6)

    print("Top_10_Most_Volatile_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("Top_10_Most_Volatile_Stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
Top_10_volatile_values =Top_10_Most_Volatile_Stocks.values.tolist()


#To store in values in the Top_10_Most_Volatile_Stocks table
try:

    insert_query_6 = ''' insert into Top_10_Most_Volatile_Stocks
                     values(%s,%s)'''
    mycursor.executemany(insert_query_6,Top_10_volatile_values)   
    mydb.commit()
    print("Top_10_Most_Volatile_Stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("Top_10_Most_Volatile_Stocks values are already exist")





#create the table for in database(comulative_return_year)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_7 = ('''CREATE TABLE comulative_return_year( Ticker VARCHAR(255), 
                                                              Cumulative_Return float)''')
    
    mycursor.execute(create_query_7)

    print("comulative_return_year TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("comulative_return_year TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
comu_values = comulative_return_year.values.tolist()


#To store in values in the comulative_return_year
try:

    insert_query_7 = ''' insert into comulative_return_year
                     values(%s,%s)'''
    mycursor.executemany(insert_query_7,comu_values)   
    mydb.commit()
    print("comulative_return_year values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("comulative_return_year values are already exist")



#create the table for in database(sector_wise_overall_return)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_8 = ('''CREATE TABLE sector_wise_overall_return( Ticker VARCHAR(255), 
                                                              	  Sector VARCHAR(255),
                                                                	Overall_Return float)''')
    
    mycursor.execute(create_query_8)

    print("sector_wise_overall_return TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("sector_wise_overall_return TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
sector_values = sector_wise_overall_return.values.tolist()

#To store in values in the comulative_return_year
try:

    insert_query_8 = ''' insert into sector_wise_overall_return
                     values(%s,%s,%s)'''
    mycursor.executemany(insert_query_8,sector_values)   
    mydb.commit()
    print("sector_wise_overall_return values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("sector_wise_overall_return values are already exist")





#create the table for in database(correlation_matrix for Each stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_9 = ('''CREATE TABLE correlation_matrix (stock_1 VARCHAR(255),
                                                          stock_2 VARCHAR(255),	
                                                          corr_values float,
                                                          Category VARCHAR(255))''')
    
    mycursor.execute(create_query_9)

    print("correlation_matrix for Each stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("correlation_matrix for Each stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
corr_values=corr_data.values.tolist()


#To store in values in the correlation_matrix for Each stocks
try:

    insert_query_9 = ''' insert into correlation_matrix
                     values(%s,%s,%s,%s)'''
    mycursor.executemany(insert_query_9,corr_values)   
    mydb.commit()
    print("correlation_matrix for Each stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("correlation_matrix for Each stocks values are already exist")




#create the table for in database(month_wise_return for Each stocks)
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Naveen@090194",
  database="stock_analysis"
)

mycursor = mydb.cursor()

try :
      
    create_query_10 = ('''CREATE TABLE month_wise_return (Ticker VARCHAR(255),
                                                        	month VARCHAR(255),
                                                         	month_return float )''')
    
    mycursor.execute(create_query_10)

    print("month_wise_return for Each stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL")
    
except :
  print("month_wise_return for Each stocks TABLES ALREADY  IN MYSQL")

# To get the values in dataframe using (.values) and converting to list
month_wise_values=month_wise_return.values.tolist()


#To store in values in the month_wise_return for Each stocks
try:

    insert_query_10 = ''' insert into month_wise_return
                     values(%s,%s,%s)'''
    mycursor.executemany(insert_query_10,month_wise_values)   
    mydb.commit()
    print("month_wise_return for Each stocks values are successfully inserted in the table")

#To avoid the duplicate entry so we can use try and except method    
except :
    print("month_wise_return for Each stocks values are already exist")





















Saved SBIN.csv
Saved BAJFINANCE.csv
Saved TITAN.csv
Saved ITC.csv
Saved TCS.csv
Saved LT.csv
Saved TATACONSUM.csv
Saved RELIANCE.csv
Saved HCLTECH.csv
Saved JSWSTEEL.csv
Saved ULTRACEMCO.csv
Saved POWERGRID.csv
Saved INFY.csv
Saved TRENT.csv
Saved BHARTIARTL.csv
Saved TATAMOTORS.csv
Saved WIPRO.csv
Saved TECHM.csv
Saved NTPC.csv
Saved HINDUNILVR.csv
Saved APOLLOHOSP.csv
Saved M&M.csv
Saved GRASIM.csv
Saved ICICIBANK.csv
Saved ADANIENT.csv
Saved ADANIPORTS.csv
Saved BEL.csv
Saved BAJAJFINSV.csv
Saved EICHERMOT.csv
Saved COALINDIA.csv
Saved MARUTI.csv
Saved INDUSINDBK.csv
Saved ASIANPAINT.csv
Saved TATASTEEL.csv
Saved HDFCLIFE.csv
Saved DRREDDY.csv
Saved SUNPHARMA.csv
Saved KOTAKBANK.csv
Saved SHRIRAMFIN.csv
Saved NESTLEIND.csv
Saved ONGC.csv
Saved CIPLA.csv
Saved BPCL.csv
Saved BRITANNIA.csv
Saved SBILIFE.csv
Saved HINDALCO.csv
Saved HEROMOTOCO.csv
Saved AXISBANK.csv
Saved HDFCBANK.csv
Saved BAJAJ-AUTO.csv


C:\Users\NAVEEN\AppData\Local\Temp\ipykernel_23800\2507924581.py:300: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sec_[row[1]] = row[0]


databse succesfully created in mysql
 stock_details TABLES ARE SUCCESFUULY CREATED IN MYSQL
 stock_details values are successfully inserted in the table
Top_10_Green_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
Top_10_Green_Stocks values are successfully inserted in the table
Top_10_Loss_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
Top_10_Loss_Stocks values are successfully inserted in the table
Avg_prices_per_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
Avg_prices_per_stocks values are successfully inserted in the table
Avg_Volume_per_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
Avg_Volume_per_stocks values are successfully inserted in the table
No_green_red_stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
No_green_red_stocks values are successfully inserted in the table
Top_10_Most_Volatile_Stocks TABLES ARE SUCCESFUULY CREATED IN MYSQL
Top_10_Most_Volatile_Stocks values are successfully inserted in the table
comulative_return_year TABLES ARE SUCCESFUULY CREATED IN MYSQL
comula